# JMS -> SmallRye Reactive Example

> Exploring incidents related to JMS to Smallrye Reactive migrations

##### Sample Applications Used
* 2 sample apps from [JBoss EAP Quickstarts](https://github.com/jboss-developer/jboss-eap-quickstarts/tree/7.4.x) were chosen:  helloworld-mdb & cmt
    * [helloworld-mdb](https://github.com/savitharaghunathan/helloworld-mdb)
    * [cmt](https://github.com/konveyor-ecosystem/cmt)

##### Using Custom Rules for JMS to SmallRye Reactive
* Rules were developed by Juanma [@jmle](https://github.com/jmle)
    * Rules originally from: https://github.com/jmle/rulesets/blob/jms-rule/default/generated/quarkus/05-jms-to-reactive-quarkus.windup.yaml

##### Background of technology being migrated JMS -> Smallrye Reactive Messaging
* Java EE:  Java Message Service (JMS)
    * https://javaee.github.io/tutorial/jms-concepts001.html
* Quarkus:  Smallrye Reactive Messaging
    * https://smallrye.io/smallrye-reactive-messaging/smallrye-reactive-messaging/3.4/index.html



# Begin Exploring
## Step #1:  Look at what we found in analysis reports
Let's look at the analysis report (with custom rules) and find what violations (Rules) are in common


In [1]:
# Install local kai package in the current Jupyter kernel
import sys

!{sys.executable} -m pip install -e ../../

Obtaining file:///Users/jmatthews/git/jwmatthews/kai
  Preparing metadata (setup.py) ... done
  Attempting uninstall: kai
    Found existing installation: kai 0.0.1
    Uninstalling kai-0.0.1:
      Successfully uninstalled kai-0.0.1
  Running setup.py develop for kai


In [37]:
import pprint

pp = pprint.PrettyPrinter(indent=2)

In [5]:
from kai.report import Report

path_cmt_analysis = "./analysis_report/cmt/output.yaml"
path_helloworld_analysis = "./analysis_report/helloworld-mdb-quarkus/output.yaml"

cmt_report = Report(path_cmt_analysis)
helloworld_report = Report(path_helloworld_analysis)

ruleset_name = "custom-ruleset"
expected_rules = [
    "jms-to-reactive-quarkus-00000",
    "jms-to-reactive-quarkus-00010",
    "jms-to-reactive-quarkus-00020",
    "jms-to-reactive-quarkus-00030",
    "jms-to-reactive-quarkus-00040",
    "jms-to-reactive-quarkus-00050",
]

Reading report from ./analysis_report/cmt/output.yaml
Reading report from ./analysis_report/helloworld-mdb-quarkus/output.yaml


In [19]:
# We can access the underlying yaml report as 'report'
cmt_report.report.keys()

dict_keys(['cloud-readiness', 'custom-ruleset', 'eap8/eap7', 'quarkus/springboot'])

In [24]:
# We want to look at the custom rules, those have a ruleset name of 'custom-ruleset'
# We can index into the YAML report with the key of the rulset name to get info for just that ruleset
cmt_report.report["custom-ruleset"].keys()

dict_keys(['name', 'description', 'violations', 'unmatched'])

In [26]:
# Once we look at the specific ruleset of interest, what we are most interested in are the 'violations'
# The violations have the matches when a given Rule was matched in the sample source code
cmt_report.report["custom-ruleset"]["violations"]

{'jms-to-reactive-quarkus-00000': {'description': 'JMS is not supported in Quarkus',
  'category': 'mandatory',
  'labels': ['konveyor.io/source=java-ee', 'konveyor.io/target=quarkus'],
  'incidents': [{'uri': 'file:///tmp/source-code/pom.xml',
    'message': "Usage of JMS is not supported in Quarkus. It is recommended to use Quarkus' SmallRye Reactive Messaging instead of JMS.\n Replace the JavaEE/Jakarta JMS dependency with Smallrye Reactive:\n \n ```\n <dependency>\n <groupId>io.quarkus</groupId>\n <artifactId>quarkus-smallrye-reactive-messaging</artifactId>\n </dependency>\n \n ```\n \n Take a look at the Smallrye Reactive Connectors link below to know more about how to interact with different technologies (AMQP, Apache Camel, ...)",
    'codeSnip': ' 68  \t\t<!-- https://access.redhat.com/maven-repository -->\n 69  \t\t<maven.redhat.repository.url>\n 70  \t\t\t${maven.repository.protocol}://maven.repository.redhat.com/ga/</maven.redhat.repository.url>\n 71  \n 72  \t\t<!-- Version

In [31]:
# Let's look to see which rules were matched for this ruleset and the CMT application
cmt_report.report["custom-ruleset"]["violations"].keys()

dict_keys(['jms-to-reactive-quarkus-00000', 'jms-to-reactive-quarkus-00010', 'jms-to-reactive-quarkus-00020', 'jms-to-reactive-quarkus-00030', 'jms-to-reactive-quarkus-00050'])

In [32]:
# Let's look to see which rules were matched for this ruleset and the HelloWorld application
helloworld_report.report["custom-ruleset"]["violations"].keys()

dict_keys(['jms-to-reactive-quarkus-00000', 'jms-to-reactive-quarkus-00010', 'jms-to-reactive-quarkus-00020', 'jms-to-reactive-quarkus-00030', 'jms-to-reactive-quarkus-00040', 'jms-to-reactive-quarkus-00050'])

In [35]:
# Let's see what Rules were matched in both applications
common_rules = set(cmt_report.report["custom-ruleset"]["violations"].keys()) & set(
    helloworld_report.report["custom-ruleset"]["violations"].keys()
)
common_rules

{'jms-to-reactive-quarkus-00000',
 'jms-to-reactive-quarkus-00010',
 'jms-to-reactive-quarkus-00020',
 'jms-to-reactive-quarkus-00030',
 'jms-to-reactive-quarkus-00050'}

In [27]:
# We also can look at ALL of the violations organized by impacted file such as with below
cmt_report.get_impacted_files()

{'src/main/java/org/jboss/as/quickstarts/cmt/ejb/CustomerManagerEJB.java': [{'ruleset_name': 'cloud-readiness',
   'violation_name': 'java-rmi-00001',
   'ruleset_description': 'This ruleset detects logging configurations that may be problematic when migrating an application to a cloud environment.',
   'violation_description': 'Java Remote Method Invocation (RMI) API',
   'message': '\n                            The Java RMI API is tightly coupled. Tight coupling is not suitable in a cloud environment because of scalability problems.\n\n                            Use a RESTful API or a message queue.\n                            ',
   'codeSnip': '  1  /*\n  2   * JBoss, Home of Professional Open Source\n  3   * Copyright 2015, Red Hat, Inc. and/or its affiliates, and individual\n  4   * contributors by the @authors tag. See the copyright.txt in the\n  5   * distribution for a full listing of individual contributors.\n  6   *\n  7   * Licensed under the Apache License, Version 2.0 (

In [36]:
print(f"CMT: How many files are impacted? {len(cmt_report.get_impacted_files())}")
print(
    f"HelloWorld: How many files are impacted? {len(helloworld_report.get_impacted_files())}"
)

CMT: How many files are impacted? 12
HelloWorld: How many files are impacted? 5
